# 1. Importando librerías

In [11]:
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2 as pg2

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, roc_curve, roc_auc_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

# 2. Leyendo archivos

In [24]:
# datos de entrenamiento
df_train = pd.read_csv('train_cupid.csv')

In [25]:
# datos de validacion
df_test = pd.read_csv('test_cupid.csv')

In [28]:
print(pd.io.sql.get_schema(df_train.reset_index(), 'train_data'))

CREATE TABLE "train_data" (
"index" INTEGER,
  "age" INTEGER,
  "height" REAL,
  "virgo" INTEGER,
  "taurus" INTEGER,
  "scorpio" INTEGER,
  "pisces" INTEGER,
  "libra" INTEGER,
  "leo" INTEGER,
  "gemini" INTEGER,
  "aries" INTEGER,
  "aquarius" INTEGER,
  "cancer" INTEGER,
  "sagittarius" INTEGER,
  "asian" INTEGER,
  "hispanic_/_latin" INTEGER,
  "black" INTEGER,
  "indian" INTEGER,
  "pacific_islander" INTEGER,
  "native_american" INTEGER,
  "middle_eastern" INTEGER,
  "colorado" INTEGER,
  "new_york" INTEGER,
  "oregon" INTEGER,
  "arizona" INTEGER,
  "hawaii" INTEGER,
  "montana" INTEGER,
  "wisconsin" INTEGER,
  "virginia" INTEGER,
  "spain" INTEGER,
  "nevada" INTEGER,
  "illinois" INTEGER,
  "vietnam" INTEGER,
  "ireland" INTEGER,
  "louisiana" INTEGER,
  "michigan" INTEGER,
  "texas" INTEGER,
  "united_kingdom" INTEGER,
  "massachusetts" INTEGER,
  "north_carolina" INTEGER,
  "idaho" INTEGER,
  "mississippi" INTEGER,
  "new_jersey" INTEGER,
  "florida" INTEGER,
  "minnesota" 

# 3. Creando base de datos e importando tablas

In [15]:
# conectando
user = 'postgres'
password = 'moeg231@'
conn_db = pg2.connect(f"user={user} password={password}")

In [16]:
# creando BD
conn_db.autocommit = True
cursor = conn_db.cursor()
cursor.execute('CREATE DATABASE apellido_nombre')
conn_db.close()

In [ ]:
# importando tablas
cursor = conn_db.cursor()
cursor.execute('CREATE TABLE "train_data" (
          "index" INTEGER,
            "age" INTEGER,
            "height" REAL,
            "virgo" INTEGER,
            "taurus" INTEGER,
            "scorpio" INTEGER,
            "pisces" INTEGER,
            "libra" INTEGER,
            "leo" INTEGER,
            "gemini" INTEGER,
            "aries" INTEGER,
            "aquarius" INTEGER,
            "cancer" INTEGER,
            "sagittarius" INTEGER,
            "asian" INTEGER,
            "hispanic_/_latin" INTEGER,
            "black" INTEGER,
            "indian" INTEGER,
            "pacific_islander" INTEGER,
            "native_american" INTEGER,
            "middle_eastern" INTEGER,
            "colorado" INTEGER,
            "new_york" INTEGER,
            "oregon" INTEGER,
            "arizona" INTEGER,
            "hawaii" INTEGER,
            "montana" INTEGER,
            "wisconsin" INTEGER,
            "virginia" INTEGER,
            "spain" INTEGER,
            "nevada" INTEGER,
            "illinois" INTEGER,
            "vietnam" INTEGER,
            "ireland" INTEGER,
            "louisiana" INTEGER,
            "michigan" INTEGER,
            "texas" INTEGER,
            "united_kingdom" INTEGER,
            "massachusetts" INTEGER,
            "north_carolina" INTEGER,
            "idaho" INTEGER,
            "mississippi" INTEGER,
            "new_jersey" INTEGER,
            "florida" INTEGER,
            "minnesota" INTEGER,
            "georgia" INTEGER,
            "utah" INTEGER,
            "washington" INTEGER,
            "west_virginia" INTEGER,
            "connecticut" INTEGER,
            "tennessee" INTEGER,
            "rhode_island" INTEGER,
            "district_of_columbia" INTEGER,
            "canada" INTEGER,
            "missouri" INTEGER,
            "germany" INTEGER,
            "pennsylvania" INTEGER,
            "netherlands" INTEGER,
            "switzerland" INTEGER,
            "mexico" INTEGER,
            "ohio" INTEGER,
            "agnosticism" INTEGER,
            "atheism" INTEGER,
            "catholicism" INTEGER,
            "buddhism" INTEGER,
            "judaism" INTEGER,
            "hinduism" INTEGER,
            "islam" INTEGER,
            "pro_dogs" REAL,
            "pro_cats" REAL,
            "spanish" INTEGER,
            "chinese" INTEGER,
            "french" INTEGER,
            "german" INTEGER,
            "single" INTEGER,
            "seeing_someone" INTEGER,
            "available" INTEGER,
            "employed" INTEGER,
            "income_between_25_50" INTEGER,
            "income_between_50_75" INTEGER,
            "income_over_75" INTEGER,
            "drugs_often" INTEGER,
            "drugs_sometimes" INTEGER,
            "drinks_not_at_all" INTEGER,
            "drinks_often" INTEGER,
            "drinks_rarely" INTEGER,
            "drinks_socially" INTEGER,
            "drinks_very_often" INTEGER,
            "orientation_gay" INTEGER,
            "orientation_straight" INTEGER,
            "sex_m" INTEGER,
            "smokes_sometimes" INTEGER,
            "smokes_trying_to_quit" INTEGER,
            "smokes_when_drinking" INTEGER,
            "smokes_yes" INTEGER,
            "body_type_overweight" INTEGER,
            "body_type_regular" INTEGER,
            "education_high_school" INTEGER,
            "education_undergrad_university" INTEGER,
            primary key ("index")
          );')